# 🔗 LangChain con LCEL - Guía Completa
## Construyendo Aplicaciones Inteligentes con IA

Este notebook te enseña a construir aplicaciones con LangChain usando **LCEL** (LangChain Expression Language).

### ¿Qué aprenderás?
1. **Chains**: Cómo conectar componentes (prompts, LLMs, parsers)
2. **Memory**: Dar memoria conversacional a tus aplicaciones
3. **Tools**: Extender capacidades del LLM con herramientas personalizadas
4. **Output Parsers**: Obtener datos estructurados del LLM
5. **RAG**: Crear sistemas que consultan tus propios documentos

### 🎯 Objetivo del notebook:
Al finalizar, sabrás crear aplicaciones de IA completas que pueden:
- Mantener conversaciones con contexto
- Usar herramientas externas (APIs, bases de datos, cálculos)
- Consultar documentos propios (PDFs, textos)
- Devolver datos estructurados (JSON, objetos validados)

## 📦 Instalación de paquetes necesarios

In [1]:
!pip install openai python-dotenv langchain-core langchain-openai langchain-community \
    pypdf faiss-cpu -q

## ⚙️ Configuración

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("⚠️ No se encontró OPENAI_API_KEY")

print("✅ Configuración cargada")

✅ Configuración cargada


---
# 🔗 PARTE 1: Chains - Conectar Componentes

## ¿Qué son las Chains?
**Chains** son secuencias de componentes conectados que procesan información paso a paso.

## ¿Para qué sirven?
Permiten crear **pipelines de procesamiento**:
- Input → Formatear → Procesar → Transformar → Output

## Componentes básicos:
1. **Prompt**: Define las instrucciones para el LLM
2. **LLM**: El modelo de IA que genera texto
3. **Parser**: Procesa la salida del LLM

## LCEL: El lenguaje para conectarlos
Usamos el operador `|` (pipe) para conectar componentes:
```python
chain = prompt | llm | parser
```

Es como una tubería: los datos fluyen de izquierda a derecha.

## 1.1 Chain Básica con LCEL

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

print("="*70)
print("🔗 CHAIN BÁSICA CON LCEL")
print("="*70)

# PASO 1: Crear el LLM (el cerebro)
print("\n1️⃣ Creando el LLM...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # Entre 0 - 1 
print("   ✅ LLM creado (gpt-4o-mini)")

# PASO 2: Crear el Prompt (las instrucciones)
print("\n2️⃣ Creando el Prompt Template...")
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un copywriter experto."),
    ("human", "Crea una descripción atractiva para: {producto}\nCaracterísticas: {caracteristicas}")
])
print("   ✅ Prompt Template creado")
print("   📝 Variables: producto, caracteristicas")

# PASO 3: Crear el Parser (procesa la salida)
print("\n3️⃣ Creando el Output Parser...")
parser = StrOutputParser()
print("   ✅ StrOutputParser creado (convierte a string)")

# PASO 4: CONECTAR TODO con el operador | (LCEL)
print("\n4️⃣ Conectando componentes con LCEL (operador |)...")
chain = prompt | llm | parser
print("   ✅ Chain creada!")

print("\n📊 FLUJO DE LA CHAIN:")
print("   Input → Prompt → LLM → Parser → Output")
print("           ↓        ↓      ↓")
print("         Formatea  Genera  Extrae")
print("                   texto   string")

# PASO 5: Ejecutar
print("\n5️⃣ Ejecutando la chain...")
resultado = chain.invoke({
    "producto": "Rotomartillo HILTI TE-70",
    "caracteristicas": "Potente, ideal para concreto, marca profesional"
})

print("\n" + "="*70)
print("🎯 DESCRIPCIÓN GENERADA:")
print("="*70)
print(resultado)
print("\n💡 Nota: Todo esto en 1 línea → chain = prompt | llm | parser")

🔗 CHAIN BÁSICA CON LCEL

1️⃣ Creando el LLM...
   ✅ LLM creado (gpt-4o-mini)

2️⃣ Creando el Prompt Template...
   ✅ Prompt Template creado
   📝 Variables: producto, caracteristicas

3️⃣ Creando el Output Parser...
   ✅ StrOutputParser creado (convierte a string)

4️⃣ Conectando componentes con LCEL (operador |)...
   ✅ Chain creada!

📊 FLUJO DE LA CHAIN:
   Input → Prompt → LLM → Parser → Output
           ↓        ↓      ↓
         Formatea  Genera  Extrae
                   texto   string

5️⃣ Ejecutando la chain...

🎯 DESCRIPCIÓN GENERADA:
Descubre la potencia imbatible del Rotomartillo HILTI TE-70, la herramienta perfecta para los profesionales que demandan rendimiento y fiabilidad en cada proyecto. Diseñado para enfrentar los desafíos más exigentes en concreto, su motor de alta eficiencia garantiza una perforación rápida y precisa, incluso en las condiciones más duras.

Con una ergonomía optimizada y un peso equilibrado, el HILTI TE-70 te permite trabajar largas horas sin fatiga,

## 1.2 Sequential Chain con LCEL

In [14]:
from langchain_core.runnables import RunnablePassthrough

print("="*70)
print("🔗 SEQUENTIAL CHAIN - Encadenar Múltiples Operaciones")
print("="*70)

print("\n💡 CONCEPTO:")
print("   Una chain ejecuta su resultado y lo pasa a la siguiente")
print("   Chain 1 → resultado → Chain 2 → resultado final")

# CHAIN 1: Generar características técnicas
print("\n1️⃣ CHAIN 1: Generar características técnicas")
prompt_caracteristicas = ChatPromptTemplate.from_messages([
    ("human", "Lista 5 características técnicas clave de: {producto}")
])
chain_caracteristicas = prompt_caracteristicas | llm | StrOutputParser()
print("   ✅ Chain 1 creada")
print("   📥 Input: producto")
print("   📤 Output: lista de características")

# CHAIN 2: Crear pitch de ventas
print("\n2️⃣ CHAIN 2: Convertir características en pitch")
prompt_pitch = ChatPromptTemplate.from_messages([
    ("human", "Convierte estas características en un pitch de ventas (máximo 50 palabras):\n{caracteristicas}")
])
chain_pitch = prompt_pitch | llm | StrOutputParser()
print("   ✅ Chain 2 creada")
print("   📥 Input: caracteristicas")
print("   📤 Output: pitch de ventas")

# COMBINAR CHAINS usando LCEL
print("\n3️⃣ COMBINANDO las 2 chains...")
print("   Sintaxis LCEL:")
print("   {\"caracteristicas\": chain1} | chain2")
print("        ↑                         ↑")
print("   Ejecuta chain1           Usa el resultado")
print("   y lo asigna a            como input de")
print("   'caracteristicas'        chain2")

chain_completa = (
    {"caracteristicas": chain_caracteristicas}
    | chain_pitch
)
print("\n   ✅ Chain secuencial creada!")

# EJECUTAR
print("\n4️⃣ EJECUTANDO...")
print("\n" + "="*60)
print("Producto: Compactador de Rodillo 524 KGS")
print("="*60)

print("\n⏳ Paso 1: Generando características...")
resultado = chain_completa.invoke({"producto": "Compactador de Rodillo 524 KGS"})

print("\n" + "="*60)
print("🎯 PITCH DE VENTAS FINAL:")
print("="*60)
print(resultado)

print("\n💡 ¿Qué pasó internamente?")
print("   1. chain_caracteristicas generó 5 características")
print("   2. El resultado se pasó automáticamente a chain_pitch")
print("   3. chain_pitch creó el pitch de ventas")
print("   4. Todo en una sola ejecución!")

🔗 SEQUENTIAL CHAIN - Encadenar Múltiples Operaciones

💡 CONCEPTO:
   Una chain ejecuta su resultado y lo pasa a la siguiente
   Chain 1 → resultado → Chain 2 → resultado final

1️⃣ CHAIN 1: Generar características técnicas
   ✅ Chain 1 creada
   📥 Input: producto
   📤 Output: lista de características

2️⃣ CHAIN 2: Convertir características en pitch
   ✅ Chain 2 creada
   📥 Input: caracteristicas
   📤 Output: pitch de ventas

3️⃣ COMBINANDO las 2 chains...
   Sintaxis LCEL:
   {"caracteristicas": chain1} | chain2
        ↑                         ↑
   Ejecuta chain1           Usa el resultado
   y lo asigna a            como input de
   'caracteristicas'        chain2

   ✅ Chain secuencial creada!

4️⃣ EJECUTANDO...

Producto: Compactador de Rodillo 524 KGS

⏳ Paso 1: Generando características...

🎯 PITCH DE VENTAS FINAL:
¡Potencia y eficiencia en tus proyectos de construcción! Nuestro compactador de 524 kg combina un rodillo vibratorio para una compactación superior, un ancho de traba

## 1.3 Parallel Chains (Ejecutar en paralelo)

In [15]:
from langchain_core.runnables import RunnableParallel

print("="*70)
print("⚡ PARALLEL CHAINS - Ejecutar Múltiples Chains AL MISMO TIEMPO")
print("="*70)

print("\n💡 CONCEPTO:")
print("   En vez de ejecutar chain1 → chain2 (secuencial)")
print("   Ejecutamos chain1 + chain2 EN PARALELO (más rápido)")
print()
print("   ┌─────────┐")
print("   │  Input  │")
print("   └────┬────┘")
print("        │")
print("   ┌────┴────┐")
print("   ▼         ▼")
print(" Chain1    Chain2")
print("   │         │")
print("   └────┬────┘")
print("        │")
print("   ┌────▼────┐")
print("   │ Results │")
print("   └─────────┘")

# CHAIN 1: Generar ventajas
print("\n1️⃣ CHAIN 1: Ventajas del producto")
prompt_ventajas = ChatPromptTemplate.from_messages([
    ("human", "Lista 3 ventajas principales de: {producto}")
])
chain_ventajas = prompt_ventajas | llm | StrOutputParser()
print("   ✅ Chain de ventajas creada")

# CHAIN 2: Generar casos de uso
print("\n2️⃣ CHAIN 2: Casos de uso")
prompt_casos = ChatPromptTemplate.from_messages([
    ("human", "Lista 3 casos de uso ideales para: {producto}")
])
chain_casos = prompt_casos | llm | StrOutputParser()
print("   ✅ Chain de casos de uso creada")

# EJECUTAR EN PARALELO usando RunnableParallel
print("\n3️⃣ Creando RunnableParallel...")
print("   Sintaxis:")
print("   RunnableParallel(")
print("       nombre1=chain1,  ← Ejecuta en paralelo")
print("       nombre2=chain2   ← Ejecuta en paralelo")
print("   )")

chain_paralela = RunnableParallel(
    ventajas=chain_ventajas,
    casos_uso=chain_casos
)
print("\n   ✅ RunnableParallel creada!")
print("   📤 Retorna: {\"ventajas\": resultado1, \"casos_uso\": resultado2}")

# EJECUTAR
print("\n4️⃣ EJECUTANDO...")
print("\n" + "="*60)
print("Producto: Rotomartillo TE-70")
print("="*60)

print("\n⚡ Ejecutando 2 chains en PARALELO...")
print("   (Esto es MÁS RÁPIDO que ejecutarlas una por una)")

import time
inicio = time.time()
resultado = chain_paralela.invoke({"producto": "Rotomartillo TE-70"})
tiempo = time.time() - inicio

print("\n" + "="*60)
print("✅ VENTAJAS:")
print("="*60)
print(resultado["ventajas"])

print("\n" + "="*60)
print("✅ CASOS DE USO:")
print("="*60)
print(resultado["casos_uso"])

print(f"\n⏱️ Tiempo de ejecución: {tiempo:.2f}segundos")
print("\n💡 Beneficio:")
print("   Si cada chain toma 2 segundos:")
print("   - Secuencial: 2 + 2 = 4 segundos")
print("   - Paralelo: máx(2, 2) = ~2 segundos (50% más rápido!)")

⚡ PARALLEL CHAINS - Ejecutar Múltiples Chains AL MISMO TIEMPO

💡 CONCEPTO:
   En vez de ejecutar chain1 → chain2 (secuencial)
   Ejecutamos chain1 + chain2 EN PARALELO (más rápido)

   ┌─────────┐
   │  Input  │
   └────┬────┘
        │
   ┌────┴────┐
   ▼         ▼
 Chain1    Chain2
   │         │
   └────┬────┘
        │
   ┌────▼────┐
   │ Results │
   └─────────┘

1️⃣ CHAIN 1: Ventajas del producto
   ✅ Chain de ventajas creada

2️⃣ CHAIN 2: Casos de uso
   ✅ Chain de casos de uso creada

3️⃣ Creando RunnableParallel...
   Sintaxis:
   RunnableParallel(
       nombre1=chain1,  ← Ejecuta en paralelo
       nombre2=chain2   ← Ejecuta en paralelo
   )

   ✅ RunnableParallel creada!
   📤 Retorna: {"ventajas": resultado1, "casos_uso": resultado2}

4️⃣ EJECUTANDO...

Producto: Rotomartillo TE-70

⚡ Ejecutando 2 chains en PARALELO...
   (Esto es MÁS RÁPIDO que ejecutarlas una por una)

✅ VENTAJAS:
El Rotomartillo TE-70 es una herramienta eléctrica popular utilizada para perforar y cincela

---
# 💭 PARTE 2: Memory - Dar Memoria a tus Aplicaciones

## ¿Qué es Memory?
Es la capacidad de **recordar conversaciones anteriores**. Sin memoria, cada interacción es independiente.

## ¿Para qué sirve?
Crear **aplicaciones conversacionales** que:
- Recuerdan el nombre del usuario
- Mantienen contexto de la conversación
- Pueden hacer seguimiento de temas anteriores

## Casos de uso:
- ✅ Chatbots de atención al cliente
- ✅ Asistentes virtuales personalizados
- ✅ Tutores educativos que recuerdan el progreso
- ✅ Sistemas de recomendación contextual

## Cómo funciona:
Guardamos el historial de mensajes (humano + IA) y lo pasamos como contexto en cada nueva interacción.

## 2.1 Memory con RunnableWithMessageHistory

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Store para guardar historiales por sesión
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Prompt con placeholder para historial
prompt_memoria = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente amable y profesional."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Chain con memoria
chain_base = prompt_memoria | llm | StrOutputParser()

chain_con_memoria = RunnableWithMessageHistory(
    chain_base,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Conversación
print("\n" + "="*60)
print("🗣️ CONVERSACIÓN CON MEMORIA")
print("="*60)

session_id = "usuario_1"
config = {"configurable": {"session_id": session_id}}

resp1 = chain_con_memoria.invoke(
    {"input": "Hola, me llamo María y necesito un demoledor"},
    config=config
)
print(f"\n🤖 {resp1}")

resp2 = chain_con_memoria.invoke(
    {"input": "¿Cuál me recomiendas?"},
    config=config
)
print(f"\n🤖 {resp2}")

resp3 = chain_con_memoria.invoke(
    {"input": "¿Recuerdas mi nombre?"},
    config=config
)
print(f"\n🤖 {resp3}")

# Ver historial
print("\n📝 HISTORIAL:")
history = get_session_history(session_id)
for msg in history.messages:
    print(f"  {msg.type}: {msg.content[:80]}...")


🗣️ CONVERSACIÓN CON MEMORIA

🤖 ¡Hola, María! Claro, puedo ayudarte con eso. ¿Estás buscando un demoledor para comprar o para alquilar? Además, ¿tienes algún tipo específico en mente, como un demoledor manual o uno eléctrico?

🤖 La elección del demoledor depende del tipo de trabajo que planeas realizar. Aquí te doy algunas recomendaciones:

1. **Demoledor eléctrico**: Ideal para trabajos en interiores o en lugares donde no tienes acceso a una fuente de combustible. Son más ligeros y silenciosos. Si vas a trabajar en proyectos pequeños o medianos, un modelo de 5 a 10 kilovatios suele ser suficiente.

2. **Demoledor de gasolina**: Estos son más potentes y son ideales para trabajos al aire libre o en lugares sin electricidad. Si necesitas romper concreto, asfalto o hacer trabajos más pesados, un modelo de este tipo sería más adecuado.

3. **Demoledor manual**: Si tienes un proyecto pequeño y no quieres invertir en un equipo pesado, un demoledor manual puede ser suficiente. Es ideal para t

## 2.2 Memory con Ventana (Trimming)

In [6]:
from langchain_core.runnables import RunnableLambda

# Función para trimear mensajes (solo últimos K)
def trim_messages(messages, k=4):
    """Mantiene solo los últimos K mensajes"""
    return messages[-k:]

# Store separado
store_ventana = {}

def get_window_history(session_id: str):
    if session_id not in store_ventana:
        store_ventana[session_id] = ChatMessageHistory()
    return store_ventana[session_id]

# Chain con ventana
chain_ventana = RunnableWithMessageHistory(
    prompt_memoria | llm | StrOutputParser(),
    get_window_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("\n" + "="*60)
print("🪟 MEMORIA CON VENTANA (k=2)")
print("="*60)

session_2 = "usuario_2"
config_2 = {"configurable": {"session_id": session_2}}

chain_ventana.invoke({"input": "Me llamo Pedro"}, config=config_2)
chain_ventana.invoke({"input": "Vivo en Honduras"}, config=config_2)
chain_ventana.invoke({"input": "Necesito un compresor"}, config=config_2)

# Trimear manualmente
history = get_window_history(session_2)
history.messages = trim_messages(history.messages, k=4)

resp = chain_ventana.invoke({"input": "¿Recuerdas de donde soy?"}, config=config_2)
print(f"\n🤖 {resp}")
print("\n💡 Solo recuerda las últimas 2 interacciones")


🪟 MEMORIA CON VENTANA (k=2)

🤖 Sí, mencionaste que vives en Honduras. Si necesitas información sobre dónde comprar un compresor en tu país o recomendaciones específicas, házmelo saber y con gusto te ayudaré.

💡 Solo recuerda las últimas 2 interacciones


---
# 🛠️ PARTE 3: Tools - Extender Capacidades del LLM

## ¿Qué son las Tools?
**Herramientas** son funciones de Python que el LLM puede usar para realizar tareas específicas.

## ¿Por qué son necesarias?
Los LLMs por sí solos tienen limitaciones:
- ❌ No pueden hacer cálculos matemáticos precisos
- ❌ No pueden consultar bases de datos
- ❌ No tienen acceso a información en tiempo real
- ❌ No pueden ejecutar código

## ¿Para qué sirven?
Las tools **extienden** las capacidades del LLM:
- ✅ Hacer cálculos exactos
- ✅ Consultar APIs externas
- ✅ Leer/escribir en bases de datos
- ✅ Verificar disponibilidad de productos
- ✅ Obtener información actualizada (clima, precios, etc.)

## Casos de uso:
- Sistema de ventas que calcula precios con descuentos
- Asistente que consulta inventario en tiempo real
- Bot que obtiene información de APIs de terceros
- Sistema que ejecuta consultas a bases de datos

## Cómo funcionan:
1. El LLM decide qué herramienta usar
2. El LLM genera los parámetros necesarios
3. Se ejecuta la herramienta
4. El resultado se le devuelve al LLM
5. El LLM genera la respuesta final al usuario

In [43]:
# import FAISS

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

VECTORSTORE_DIR = "vectorstore_db"
EMBEDDING_MODEL = "text-embedding-3-small"
TOP_K_DOCUMENTS = 3

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    
    
vectorstore = FAISS.load_local(VECTORSTORE_DIR, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": TOP_K_DOCUMENTS}
    )

## 3.1 Definir Tools con Pydantic

In [44]:
from langchain_core.tools import tool
from datetime import datetime, timedelta

@tool
def calcular_descuento(precio: float, dias: int) -> dict:
    """Calcula el precio total con descuentos por volumen.
    
    Args:
        precio: Precio por día en Lempiras
        dias: Número de días de renta
    """
    total = precio * dias
    
    if dias >= 30:
        descuento = 0.20
    elif dias >= 14:
        descuento = 0.15
    elif dias >= 7:
        descuento = 0.10
    else:
        descuento = 0
    
    total_con_descuento = total * (1 - descuento)
    ahorro = total - total_con_descuento
    
    return {
        "total_sin_descuento": f"L{total:.2f}",
        "descuento_porcentaje": f"{descuento*100}%",
        "total_con_descuento": f"L{total_con_descuento:.2f}",
        "ahorro": f"L{ahorro:.2f}"
    }

@tool
def verificar_disponibilidad(equipo: str) -> str:
    """Verifica si un equipo está disponible en inventario.
    
    Args:
        equipo: Nombre del equipo
    """
    inventario = {
        "demoledor": {"disponible": True, "unidades": 3},
        "rotomartillo": {"disponible": True, "unidades": 5},
        "compactador": {"disponible": False, "unidades": 0}
    }
    
    for key, info in inventario.items():
        if key in equipo.lower():
            if info["disponible"]:
                return f"✅ {equipo} disponible. Stock: {info['unidades']} unidades"
            else:
                return f"❌ {equipo} agotado. Stock: {info['unidades']}"
    
    return f"⚠️ No encontrado: {equipo}"

@tool
def calcular_fecha_entrega(dias: int) -> str:
    """Calcula la fecha de devolución del equipo.
    
    Args:
        dias: Número de días de renta
    """
    
    tiempo_transporte = 1  # días adicionales por transporte
    dias += tiempo_transporte
    
    fecha = datetime.now() + timedelta(days=dias)
    return f"Fecha de devolución: {fecha.strftime('%d/%m/%Y')}"

@tool
def buscar_info_producto(producto: str) -> str:
    """Busca información de productos en el catálogo (precios, especificaciones, etc).
    
    Args:
        producto: Nombre del producto a buscar
    """
    # Usa invoke() en lugar de get_relevant_documents()
    docs = retriever.invoke(producto)
    if docs:
        return docs[0].page_content[:500]
    return "No se encontró información del producto"
    
tools = [calcular_descuento, verificar_disponibilidad, calcular_fecha_entrega, buscar_info_producto]

print("✅ Tools creadas con decorador @tool")
print(f"\nHerramientas disponibles: {[t.name for t in tools]}")

✅ Tools creadas con decorador @tool

Herramientas disponibles: ['calcular_descuento', 'verificar_disponibilidad', 'calcular_fecha_entrega', 'buscar_info_producto']


## 3.2 LLM con Function Calling (Forma Moderna)

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool

# Bind tools al LLM (OpenAI Function Calling)
llm_con_tools = llm.bind_tools(tools)

# Ejemplo de uso
print("\n" + "="*60)
print("🤖 LLM CON FUNCTION CALLING")
print("="*60)

# El LLM decide qué herramienta usar
#response = llm_con_tools.invoke("Necesito el rotomartillo para el martes próximo")
response = llm_con_tools.invoke("Calcula el precio del demoledor para 30 días")
#response = llm_con_tools.invoke("Si rento por 10 días, ¿cuándo debo devolverlo?")
if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"  Herramienta: {tool_call['name']}")
        print(f"  Argumentos: {tool_call['args']}")
        
        # Ejecutar la herramienta
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        # Buscar la tool correspondiente
        for tool in tools:
            if tool.name == tool_name:
                result = tool.invoke(tool_args)
                print(f"  Resultado: {result}")
else:
    print("  (No se llamó ninguna herramienta)")
    print(f"  Respuesta directa: {response.content}")


🤖 LLM CON FUNCTION CALLING
  Herramienta: calcular_fecha_entrega
  Argumentos: {'dias': 10}
  Resultado: Fecha de devolución: 17/12/2025


## 3.3 Agent Loop Manual

In [47]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def ejecutar_agente(pregunta: str, max_iterations: int = 5):
    """Agente simple que usa tools sin langchain-classic"""
    
    messages = [HumanMessage(content=pregunta)]
    
    for i in range(max_iterations):
        print(f"\n🔄 Iteración {i+1}")
        
        # LLM decide
        response = llm_con_tools.invoke(messages)
        messages.append(response)
        
        # Si no hay tool calls, terminamos
        if not response.tool_calls:
            print(f"✅ Respuesta final: {response.content}")
            return response.content
        
        # Ejecutar tools
        for tool_call in response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            
            print(f"🔧 Usando: {tool_name}({tool_args})")
            
            # Buscar y ejecutar tool
            for tool in tools:
                if tool.name == tool_name:
                    result = tool.invoke(tool_args)
                    print(f"   Resultado: {result}")
                    
                    # Agregar resultado a mensajes
                    messages.append(
                        ToolMessage(
                            content=str(result),
                            tool_call_id=tool_call['id']
                        )
                    )
    
    return "Max iteraciones alcanzadas"

# Probar el agente
print("\n" + "="*60)
print("🤖 AGENTE MANUAL (SIN LANGCHAIN-CLASSIC)")
print("="*60)

resultado = ejecutar_agente(
    "¿Cuánto cuesta rentar un demoledor por 15 días, que fecha de entrega tendria?"
)


🤖 AGENTE MANUAL (SIN LANGCHAIN-CLASSIC)

🔄 Iteración 1
🔧 Usando: buscar_info_producto({'producto': 'demoledor'})
   Resultado: Disponibilidad: En stock
DEMOLEDOR TE-800
Precio: L550.00 por dia
Martillo demoledor de alto rendimiento y robusto para trabajar en paredes y pisos. Marca HILTI modelo TE-800.
Disponibilidad: En stock
DEMOLEDOR TE-1000
Precio: L800.00 por dia
Martillo rompedor versatil para tareas de rotura de suelos y aplicaciones ocasionales de paredes. Marca HILTI modelo
TE-1000.
Disponibilidad: En stock
DEMOLEDOR TE-2000
🔧 Usando: calcular_fecha_entrega({'dias': 15})
   Resultado: Fecha de devolución: 22/12/2025

🔄 Iteración 2
🔧 Usando: calcular_descuento({'precio': 550, 'dias': 15})
   Resultado: {'total_sin_descuento': 'L8250.00', 'descuento_porcentaje': '15.0%', 'total_con_descuento': 'L7012.50', 'ahorro': 'L1237.50'}

🔄 Iteración 3
✅ Respuesta final: Para rentar un **demoledor** (modelo TE-800) por **15 días** el costo sería:

- **Precio sin descuento:** L8250.00
- **D

---
# 📊 PARTE 4: Output Parsers - Estructurar Datos

## ¿Qué son los Output Parsers?
Componentes que **convierten** la respuesta de texto del LLM en **datos estructurados**.

## ¿Cuál es el problema?
Por defecto, el LLM devuelve texto libre:
```
"El producto cuesta L750 por día y está disponible"
```

Esto es difícil de procesar programáticamente.

## ¿Para qué sirven?
Obtener **datos estructurados** que puedes usar en tu aplicación:
```json
{
  "precio": 750,
  "disponible": true,
  "moneda": "L"
}
```

## Casos de uso:
- ✅ Extraer información de productos (nombre, precio, stock)
- ✅ Clasificar texto (categorías, sentimientos)
- ✅ Generar formularios estructurados
- ✅ Crear datos para bases de datos
- ✅ Validar tipos de datos

## Tipos de parsers:
1. **JsonOutputParser**: Convierte a JSON/diccionarios
2. **PydanticOutputParser**: Valida tipos con Pydantic
3. **StructuredOutput**: Usa función nativa de OpenAI (más confiable)

## 4.1 JSON Output Parser

In [41]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class Producto(BaseModel):
    nombre: str = Field(description="Nombre del producto")
    precio: str = Field(description="Precio en Lempiras")
    categoria: str = Field(description="Categoría")
    disponible: bool = Field(description="Disponibilidad")

parser = JsonOutputParser(pydantic_object=Producto)

prompt_json = ChatPromptTemplate.from_messages([
    ("system", "Extrae información y devuelve JSON.\n{format_instructions}"),
    ("human", "{text}")
])

chain_json = (
    prompt_json.partial(format_instructions=parser.get_format_instructions())
    | llm
    | parser
)

resultado = chain_json.invoke({
    "text": "Rotomartillo TE-70, L750/día, herramientas, en stock"
})

print("\n📊 SALIDA JSON:")
print(resultado)
print(f"\nNombre: {resultado['nombre']}")
print(f"Disponible: {resultado['disponible']}")


📊 SALIDA JSON:
{'nombre': 'Rotomartillo TE-70', 'precio': 'L750/día', 'categoria': 'herramientas', 'disponible': True}

Nombre: Rotomartillo TE-70
Disponible: True


## 4.2 Structured Output (OpenAI Native)

In [42]:
from typing import List

class ProductoCompleto(BaseModel):
    """Información completa de un producto"""
    nombre: str = Field(description="Nombre del producto")
    precio: float = Field(description="Precio por día en Lempiras")
    caracteristicas: List[str] = Field(description="Lista de características")
    disponible: bool = Field(description="Si está disponible")
    categoria: str = Field(description="Categoría del producto")

# Usar structured output nativo de OpenAI
llm_structured = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
).with_structured_output(ProductoCompleto)

prompt_structured = ChatPromptTemplate.from_messages([
    ("human", "Extrae información del producto: {descripcion}")
])

chain_structured = prompt_structured | llm_structured

producto = chain_structured.invoke({
    "descripcion": "Demoledor TE-3000 HILTI, martillo rompedor potente para concreto pesado, L1100 por día, disponible en stock"
})

print("\n🎯 OBJETO PYDANTIC:")
print(f"Nombre: {producto.nombre}")
print(f"Precio: L{producto.precio}")
print(f"Características: {producto.caracteristicas}")
print(f"Categoría: {producto.categoria}")
print(f"Disponible: {producto.disponible}")


🎯 OBJETO PYDANTIC:
Nombre: Demoledor TE-3000 HILTI
Precio: L1100.0
Características: ['Martillo rompedor potente', 'Diseñado para concreto pesado', 'Ideal para trabajos de demolición', 'Fácil de manejar', 'Durabilidad garantizada']
Categoría: Herramientas de construcción
Disponible: True


---
# 📚 Resumen del Notebook

## ✅ Lo que aprendiste:

### 1. **Chains con LCEL**
- Conectar componentes con el operador `|`
- Crear pipelines de procesamiento
- Ejecutar chains secuenciales y paralelas

**Sintaxis clave:**
```python
chain = prompt | llm | parser
resultado = chain.invoke(input)
```

### 2. **Memory**
- Dar memoria conversacional a aplicaciones
- Mantener contexto entre interacciones
- Implementar diferentes estrategias (ventana, resumen)

**Sintaxis clave:**
```python
chain_con_memoria = RunnableWithMessageHistory(chain, get_history)
```

### 3. **Tools**
- Extender capacidades del LLM
- Crear funciones personalizadas
- Usar Function Calling de OpenAI

**Sintaxis clave:**
```python
@tool
def mi_herramienta(param: tipo) -> tipo:
    """Descripción de qué hace"""
    return resultado

llm_con_tools = llm.bind_tools([mi_herramienta])
```

### 4. **Output Parsers**
- Obtener datos estructurados del LLM
- Validar tipos con Pydantic
- Usar structured output nativo

**Sintaxis clave:**
```python
parser = JsonOutputParser(pydantic_object=MiClase)
chain = prompt | llm | parser
```

---

## 🚀 Próximos Pasos

### Practica creando:
1. Un chatbot con memoria para tu negocio
2. Un sistema RAG sobre tus propios documentos
3. Un asistente con tools personalizadas
4. Una aplicación que combine todo lo anterior

### Recursos adicionales:
- [Documentación de LangChain](https://python.langchain.com/)
- [Cookbook con ejemplos](https://github.com/langchain-ai/langchain/tree/master/cookbook)
- [Comunidad en Discord](https://discord.gg/langchain)

---

## 💡 Consejos Finales

### ✅ Mejores prácticas:
1. **Empieza simple**: Primero haz que funcione, luego optimiza
2. **Prueba iterativamente**: Ejecuta cada componente por separado
3. **Usa verbose**: Ayuda a debuggear (`llm.invoke(..., verbose=True)`)
4. **Documenta tus tools**: El LLM usa las descripciones para decidir
5. **Monitorea costos**: Usa callbacks para trackear tokens

### ⚠️ Errores comunes:
1. Descripciones de tools poco claras → El LLM no sabe cuándo usarlas
2. Chunks muy grandes en RAG → Excede límite de contexto
3. No validar inputs de tools → Errores en runtime
4. Olvidar formatear contexto en RAG → El LLM recibe mal los docs
5. No limitar iteraciones en loops → Puede quedar en loop infinito

---

**¡Éxito en tus proyectos con LangChain! 🎓**